# Setup

In [23]:
from openai import OpenAI
import os, json, itertools
from IPython.display import display, Markdown


In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')

In [26]:
# Connect to the OpenAI api
client = OpenAI()

# Define the inputs

In [27]:
# Define the model inputs
MODEL = "gpt-4.1"
MODEL_MINI = "gpt-4.1-mini"
TOOLS = [{ "type": "web_search" }]

In [28]:
# Developer message definition
developer_message = """
You are an expert Deep Researcher.
You provide complete and in depth research to the user.
"""


In [29]:
# Request the topic of research
topic = input("Please enter the research topic: ")
topic

'Shiba Inu'

# LLM asks questions

In [32]:
# Define here the prompt to clarify
prompt_to_clarify = f"""
Ask 5 numbered clarifying question to the user about the topic: {topic}.
The goal of the questions is to understand the intended purpose of the research and help the user to understand it.
Reply only with the questions
"""

In [33]:
# Get the OpenAI API to ask 5 clarifying questions
clarify = client.responses.create(
    model = MODEL_MINI,
    input = prompt_to_clarify,
    instructions = developer_message
)

In [34]:
# Get the questions
questions = clarify.output[0].content[0].text.split( "\n")
questions

['1. Are you interested in researching the Shiba Inu dog breed, the cryptocurrency Shiba Inu (SHIB), or both?  ',
 '2. What specific aspects of Shiba Inu are you looking to explore (e.g., history, behavior, training for the dog breed; market performance, technology, investment strategies for the cryptocurrency)?  ',
 '3. What is the intended purpose of your research—academic study, investment decision, pet ownership preparation, or something else?  ',
 '4. Do you require detailed technical information, or would a general overview suffice?  ',
 '5. Are you looking for recent developments and trends, or a comprehensive historical background?']

In [37]:
# Ask each question to the user
answers = []
for question in questions:
  answer = input(question)
  answers.append(answer)

# Web searches

## Goal and Queries

In [38]:
# Write the prompt_goals prompt
prompt_goals = f"""
Using the user answers {answers} to the questions {questions}, write a goal sentence and 5 web search queries for the research about {topic}
Output: A json list of the goal and the 5 web search queries that will reach it.
Format: {{\"goal\": \"...\", \"queries\": [\"q1\", ....]}}
"""

In [39]:
# Use the responses API
goal_and_queries = client.responses.create(
    model = MODEL,
    input = prompt_goals,
    previous_response_id = clarify.id,
    instructions = developer_message
)
goal_and_queries.output[0].content[0].text

'{\n  "goal": "To provide a comprehensive, technical understanding of Shiba Inu dog breed behavior to prepare prospective pet owners for successful ownership.",\n  "queries": [\n    "technical analysis of Shiba Inu dog breed behavior",\n    "comprehensive guide to Shiba Inu behavior traits and temperament",\n    "Shiba Inu dog behavioral challenges and solutions for new owners",\n    "scientific research on Shiba Inu breed-specific behavior",\n    "how to prepare for Shiba Inu ownership: technical and behavioral considerations"\n  ]\n}'

In [40]:
# Formatting and loading as JSON
plan = json.loads(goal_and_queries.output[0].content[0].text)

In [41]:
# Check the goal and the queries
print(plan["goal"])
print(plan["queries"])

To provide a comprehensive, technical understanding of Shiba Inu dog breed behavior to prepare prospective pet owners for successful ownership.
['technical analysis of Shiba Inu dog breed behavior', 'comprehensive guide to Shiba Inu behavior traits and temperament', 'Shiba Inu dog behavioral challenges and solutions for new owners', 'scientific research on Shiba Inu breed-specific behavior', 'how to prepare for Shiba Inu ownership: technical and behavioral considerations']


In [42]:
# Store the goal and queries
goal = plan["goal"]
queries = plan["queries"]

## Web search

In [43]:
# Print the first query
print(queries[0])

technical analysis of Shiba Inu dog breed behavior


In [46]:
# Web search with the the responses endpoint
web_search = client.responses.create(
    model = MODEL,
    input = f"search: {queries[0]}",
    instructions = developer_message,
    tools = [{"type":"web_search"}]
)

In [47]:
# Investigate the output
# Retrieve the id
web_search.output[1].id


'msg_687d91abcda0819896d372175b8d1c9a0af3885bb17abaed'

In [48]:
web_search.output[1].content[0].text

'The Shiba Inu, Japan\'s smallest native dog breed, exhibits a complex behavioral profile characterized by independence, intelligence, and a strong-willed nature. Understanding these traits is essential for effective training and harmonious cohabitation.\n\n**Independence and Strong Will**\n\nShiba Inus are renowned for their independent and confident demeanor. Often likened to cats, they value autonomy and may display aloofness, especially towards strangers. This independence can manifest as stubbornness, making training a challenge. Consistent, patient, and positive reinforcement techniques are crucial to navigate their strong-willed nature. ([tailwise.co](https://tailwise.co/10-essential-traits-of-the-shiba-inu-breed/?utm_source=openai))\n\n**Intelligence and Curiosity**\n\nThese dogs possess sharp intellect and keen observational skills, enabling them to learn quickly. However, their intelligence, coupled with an independent streak, can lead to selective obedience. Engaging them in

In [49]:
# Build the function for the web searches
def run_search(q):
  web_search = client.responses.create(
    model = MODEL,
    input = f"search: {q}",
    instructions = developer_message,
    tools = TOOLS
  )
  return {"query": q,
          "resp_id": web_search.output[1].id,
          "research_output": web_search.output[1].content[0].text}


In [50]:
# Test the funtion
run_search(queries[0])

{'query': 'technical analysis of Shiba Inu dog breed behavior',
 'resp_id': 'msg_687d91c1be3c819a861a7a3a1d55cac903b32958d29c2e66',
 'research_output': 'The Shiba Inu, Japan\'s smallest native dog breed, exhibits a complex behavioral profile characterized by independence, intelligence, and a strong-willed nature. Understanding these traits is essential for effective training and harmonious cohabitation.\n\n**Independence and Strong Will**\n\nShiba Inus are renowned for their independent and confident demeanor. This self-reliance often translates into stubbornness, making them less inclined to follow commands without question. Their independent streak can be traced back to their origins as hunting dogs in Japan, where they were bred to work independently and make decisions on their own. ([tailwise.co](https://tailwise.co/10-essential-traits-of-the-shiba-inu-breed/?utm_source=openai))\n\n**Intelligence and Curiosity**\n\nThese dogs possess high intelligence and a keen sense of curiosity,

# Verifying if we have enough information to reach the goal

In [51]:
# Create a function that evaluates the outcome
def evaluate(collected):
  review = client.responses.create(
      model = MODEL,
      input = [
          {"role": "developer", "content": f"Research goal: {goal}"},
          {"role": "assistant", "content": json.dumps(collected)},
          {"role": "user", "content": "Does this information will fully satisfy the goal? Answer Yes or No only."}
      ],
      instructions = developer_message
  )
  return "yes" in review.output[0].content[0].text.lower()

In [52]:
# Verifying if we have enough info
collected = []
for _ in itertools.count():
  for q in queries:
    collected.append(run_search(q))
  if evaluate(collected):
    break

  # If no: perform 5 more questions
  more_searches = client.responses.create(
      model = MODEL,
      input = [
          {"role": "assistant", "content": f"Current data: {json.dumps(collected)}"},
          {"role": "user", "content": f"This has not met the goal: {goal}. Write 5 other web searchs to achieve the goal"}
      ],
      instructions = developer_message,
      previous_response_id = goal_and_queries.id
  )
  queries = json.loads(more_searches.output[0].content[0].text)


KeyboardInterrupt: 

# Write the final report

In [53]:
# Use the responses endpoint to write the deep research report
report = client.responses.create(
    model = MODEL,
    input = [
        {"role": "developer", "content": (f"Write a complete and detailed report about research goal: {goal}"
                                        "Cite Sources inline using [n] and append a reference"
                                        "list mapping [n] to url")},
        {"role": "assistant", "content": json.dumps(collected)}],
    instructions = developer_message
)
print("\n=== FINAL REPORT ===")
display(Markdown(report.output[0].content[0].text))


=== FINAL REPORT ===


# Comprehensive Technical Report: Understanding Shiba Inu Dog Breed Behavior for Prospective Pet Owners

---

## Executive Summary

The Shiba Inu is a Japanese breed known for intelligence, independence, and a spirited temperament. Prospective owners must thoroughly understand Shiba Inu behavior to set realistic expectations and implement strategies that support successful ownership. This detailed report synthesizes current research on the breed's behavioral traits, training challenges, temperament, and best practices for novice and experienced owners alike.

---

## 1. Overview of the Shiba Inu Breed

Shiba Inus are the smallest and one of the oldest Japanese breeds, originally developed as agile hunters in mountainous regions. Their most recognizable features are their fox-like appearance, erect ears, curled tails, and compact stature. Shibas possess a broad suite of behavioral characteristics rooted in their hunting history and centuries of selective breeding.

---

## 2. Key Behavioral Traits

### Independence and Willfulness

Shiba Inus are famously independent, sometimes compared behaviorally to cats rather than typical companion dogs. Their autonomy manifests as:

- **Selective Attention**: They may ignore commands or tasks unless motivated.
- **Self-Sufficiency**: Often content being alone, less prone to separation anxiety [1].
- **Stubbornness**: May challenge owners to maintain consistency and assertiveness in training [2][3].

### Intelligence and Problem-Solving

The Shiba Inu is a highly intelligent breed:
- **Quick Learners**: Excel at learning new commands; however, their willful nature can impede consistent compliance [1].
- **Curiosity**: React rapidly to changes in their environment and enjoy mental challenges [1].

### Loyalty and Social Bonds

While reserved with strangers, Shibas form deep attachments to their families:
- **Subtle Affection**: Display loyalty through proximity, following owners, and indirect attention.
- **Reserved Demeanor**: Rarely display exuberant affection, preferring “on their own terms” interaction [1][2].

### Prey Drive and Territoriality

Bred as hunters, Shiba Inus have:
- **Strong Prey Drive**: Chasing small animals is instinctual, necessitating vigilance in unfenced areas.
- **Possessiveness**: May guard toys, food, or people, especially without early intervention [1][3].
- **Territorial Alertness**: Act as keen watchdogs, alerting owners to the presence of unfamiliar humans or animals [1].

### Vocalizations

Shibas are not excessive barkers, but have a unique, high-pitched “Shiba scream” when distressed or excited [1].

### Cleanliness

- **Self-Grooming**: Fastidiously clean themselves akin to cats, and are relatively easy to housebreak [1][2].
- **Seasonal Shedding**: Their double coat requires regular maintenance, especially during biannual shedding periods [1].

---

## 3. Behavioral Challenges and Management Strategies

### Stubbornness and Training Resistance

- **Challenge**: Reluctance to follow commands without appropriate motivation and consistency [2][3].
- **Solution**: Use positive reinforcement; high-value treats, play, and praise are effective. Avoid harsh corrections, which reduce trust [4].

### Aggression and Resource Guarding

- **Challenge**: Potential aggression towards other dogs and possessiveness over food/toys [2][3].
- **Solution**: Early, controlled socialization and behavior modification techniques are crucial. Brief, repeated exposures to new people/animals foster confidence and reduce guarding tendencies [5].

### Escape Attempts and “Bolting”

- **Challenge**: High prey drive and curiosity can lead to attempts to flee yards or bolt through open doors [2].
- **Solution**: Secure fencing and diligent leash training are mandatory. Utilize recall training with escalating distractions [4][6].

### Destructive Behaviors

- **Challenge**: Without adequate physical and mental stimulation, Shibas may become destructive (chewing, digging, etc.) [3].
- **Solution**: Provide daily exercise, puzzle toys, training games, and short, varied activities to maintain engagement [5].

---

## 4. Exercise, Socialization, and Training

### Exercise Requirements

- **Medium to High Activity**: Shibas need at least one hour of vigorous activity daily—walks, play, or agility [7].
- **Mental Challenges**: Incorporate training sessions, scent work, or puzzle feeders to stave off boredom [5].

### Socialization

- **Early Socialization**: Expose puppies to a variety of people, environments, and other animals before 16 weeks of age. This critical period reduces lifelong reactivity and stranger aggression [5].

### Training Approaches

- **Consistent Routine**: Use concise verbal cues, reward-based methods, and predictable scheduling.
- **Short Sessions**: Professional trainers recommend multiple brief (5-10 minute) sessions daily rather than long, repetitive training [4].
- **Professional Help**: Consider hiring a breed-savvy trainer for persistent behavioral issues [3].

---

## 5. Compatibility with Families and Other Pets

- **Children**: Shibas can do well with respectful, older children. Supervision is important to prevent resource-guarding or accidental provocation [2].
- **Other Dogs/Pets**: With early socialization, they often coexist peacefully; however, same-sex aggression and high prey drive toward small pets may persist [1][5].

---

## 6. Conclusion and Recommendations

The Shiba Inu is an intelligent, active, and independent breed that demands consistency, patience, and creativity from its owners. Prospective Shiba owners should carefully consider their ability to provide early socialization, daily physical and mental stimulation, and positive, structured training. In return, owners will gain a loyal, unique, and rewarding companion.

---

## References

[1] iheartdogs.com. “Shiba Inu Temperament: What’s a Shiba Inu’s Personality Like?”  
https://iheartdogs.com/shiba-inu-temperament-whats-a-shiba-inus-personality-like/?utm_source=openai

[2] The Spruce Pets. “Shiba Inu Dog Breed Profile.”  
https://www.thesprucepets.com/shiba-inu-dog-breed-profile-4775761?utm_source=openai

[3] Petcurious.com. “Shiba Inu Dog Breed Information.”  
https://www.petcurious.com/dog-breeds/shiba-inu/?utm_source=openai

[4] SmartDogLearning.com. “Shiba Inu Training: Tips and Techniques.”  
https://smartdoglearning.com/shiba-inu-training/?utm_source=openai

[5] ShibaInuMasters.com. “Tips for Training Shiba Inu Puppies.”  
https://shibainumasters.com/tips-for-training-shiba-inu-puppies/?utm_source=openai

[6] Colorado Shiba Inu Rescue. “Shiba 101.”  
https://coloradoshibainurescue.org/about-shiba-inus/shiba-101/?utm_source=openai

[7] American Kennel Club. “Is the Shiba Inu the Right Dog for You?”  
https://www.akc.org/expert-advice/dog-breeds/shiba-inu-right-for-you/?utm_source=openai

---
This reference list maps [n] used inline to URLs for further reading and validation.

In [1]:
!pip show openai

Name: openai
Version: 1.97.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /Users/annikamcmullen/Downloads/OpenAI - Deep Research Clone/.venv/lib/python3.13/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
